# Title and summary

NeurIPS 2025 Polymer Prediction — GNN Head + Blend with Non‑GNN Stack

This notebook builds molecular graphs from SMILES, trains a GNN regressor for five targets, produces out‑of‑fold and test predictions, and exports them for blending with the non‑GNN stack. The goal is complementary signal: the GNN alone may trail tuned GBMs on some targets, but blending typically lifts leaderboard score.

# Imports and setup

In [ ]:
# --- Cell 1: All Imports and Setup ---
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import os
import gc
import copy

# PyTorch and PyG
import torch
import torch.nn.functional as F
from torch.nn import Module, Linear, BatchNorm1d, ReLU, Dropout, Sequential
import torch.optim as optim
from torch.utils.data import Subset
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATv2Conv, global_mean_pool, global_add_pool

# RDKit
from rdkit import Chem, DataStructs
from rdkit.Chem import Descriptors, AllChem
from rdkit.Chem.rdchem import HybridizationType

# ML Models and Tools
from sklearn.model_selection import KFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import ElasticNetCV
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

# --- Global Configuration ---
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
targets = ["Tg", "FFV", "Rg", "Density", "Tc"]

# --- Environment Setup ---
if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed_all(SEED)
else:
    device = torch.device("cpu")
print(f"Using device: {device}")
print("✅ Cell 1 Complete: All setup is ready.")

# Data loading

Loads official train/test and any supplements used; deduplicates on SMILES; prepares target matrix with per‑target masks. Ensures splits align with non‑GNN folds for consistent blending later.

In [ ]:
# --- Cell 2: Data Loading ---

DATA_ROOT = "/kaggle/input/neurips-open-polymer-prediction-2025"
train = pd.read_csv(f"{DATA_ROOT}/train.csv")
test  = pd.read_csv(f"{DATA_ROOT}/test.csv")

# Optional supplements (uncomment if used in your run)
supp1 = pd.read_csv(f"{DATA_ROOT}/train_supplement/dataset1.csv")
supp3 = pd.read_csv(f"{DATA_ROOT}/train_supplement/dataset3.csv")
supp4 = pd.read_csv(f"{DATA_ROOT}/train_supplement/dataset4.csv")

# Merge supplements and deduplicate on SMILES
aug = pd.concat([supp1, supp3, supp4], ignore_index=True)
train_full = pd.concat([train, aug], ignore_index=True)
train_full = train_full.drop_duplicates(subset=["SMILES"]).reset_index(drop=True)

y_df = train_full[targets].copy()
smiles_train = train_full["SMILES"].tolist()
smiles_test  = test["SMILES"].tolist()

print(f"Train rows (deduped): {len(train_full)} | Test rows: {len(test)}")

# Graph featurization

Converts SMILES to molecular graphs:

* Atom features: element one‑hot, degree, formal charge, aromaticity, hybridization, implicit Hs.

* Bond features: bond type one‑hot, conjugation, ring membership.
Graphs stored as torch_geometric Data with x, edge_index, edge_attr.

In [ ]:
# --- Cell 3: Graph Builder Utilities ---

# Minimal periodic table set; expand if needed
ELEMENTS = ["H","C","N","O","F","P","S","Cl","Br","I"]
E2I = {e:i for i,e in enumerate(ELEMENTS)}

def atom_features(atom):
    feats = []
    # Element one-hot (truncate/other -> all zeros except a catch-all if desired)
    elem = [0]*len(ELEMENTS)
    elem[E2I.get(atom.GetSymbol(), 0)] = 1
    feats += elem
    # Scalar/categorical encodings
    feats += [
        atom.GetDegree(),
        atom.GetFormalCharge(),
        int(atom.GetIsAromatic()),
        atom.GetTotalNumHs(includeNeighbors=True),
    ]
    # Hybridization one-hot (common set)
    hyb_list = [
        HybridizationType.SP, HybridizationType.SP2,
        HybridizationType.SP3, HybridizationType.SP3D
    ]
    hyb = [int(atom.GetHybridization()==h) for h in hyb_list]
    feats += hyb
    return torch.tensor(feats, dtype=torch.float)

def bond_features(bond):
    if bond is None:
        # For self-loops if added; keep zero vector
        return torch.zeros(6, dtype=torch.float)
    btype = [
        int(bond.GetBondType().name == "SINGLE"),
        int(bond.GetBondType().name == "DOUBLE"),
        int(bond.GetBondType().name == "TRIPLE"),
        int(bond.GetBondType().name == "AROMATIC"),
    ]
    conj  = int(bond.GetIsConjugated())
    inrng = int(bond.IsInRing())
    return torch.tensor(btype + [conj, inrng], dtype=torch.float)

def smiles_to_data(smiles: str):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        # Return empty graph fallback
        return Data(x=torch.zeros((1, len(ELEMENTS)+4+4), dtype=torch.float),
                    edge_index=torch.zeros((2,0), dtype=torch.long),
                    edge_attr=torch.zeros((0,6), dtype=torch.float))
    atoms = [atom_features(a) for a in mol.GetAtoms()]
    x = torch.stack(atoms, dim=0) if atoms else torch.zeros((1, len(ELEMENTS)+4+4))

    edges_src, edges_dst, eattrs = [], [], []
    for bond in mol.GetBonds():
        u = bond.GetBeginAtomIdx()
        v = bond.GetEndAtomIdx()
        bf = bond_features(bond)
        # Undirected as two directed edges
        edges_src += [u, v]
        edges_dst += [v, u]
        eattrs += [bf, bf]
    if len(edges_src) == 0:
        edge_index = torch.zeros((2,0), dtype=torch.long)
        edge_attr  = torch.zeros((0,6), dtype=torch.float)
    else:
        edge_index = torch.tensor([edges_src, edges_dst], dtype=torch.long)
        edge_attr  = torch.stack(eattrs, dim=0)
    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

# Dataset and DataLoader

Creates pytorch‑geometric Dataset that builds graphs lazily from SMILES, caches in memory, and exposes indices mapping back to original rows. Train/test loaders use reasonable batch sizes.

In [ ]:
# --- Cell 4: Dataset and DataLoaders ---

class SmilesGraphDataset(Dataset):
    def __init__(self, smiles_list, y=None, transform=None, pre_transform=None):
        super().__init__(None, transform, pre_transform)
        self.smiles_list = smiles_list
        self.y = y.values if isinstance(y, pd.DataFrame) else y

    def len(self):
        return len(self.smiles_list)

    def get(self, idx):
        s = self.smiles_list[idx]
        g = smiles_to_data(s)
        g.idx = idx
        if self.y is not None:
            # y shape: 5 targets with NaN possible
            g.y = torch.tensor(self.y[idx], dtype=torch.float)
        return g

BATCH_SIZE = 128

full_train_ds = SmilesGraphDataset(smiles_train, y_df)
test_ds       = SmilesGraphDataset(smiles_test, None)

print(f"Graphs: train={full_train_ds.len()} | test={test_ds.len()}")

# GNN model

Defines a lightweight GATv2 encoder with edge attributes concatenation through linear mixing; global mean pooling; a shared MLP trunk; and five regression heads for multi‑task outputs.

In [ ]:
# --- Cell 5: GNN Model Definition ---

class GATBackbone(Module):
    def __init__(self, in_dim, edge_dim, hid=128, layers=3, heads=2, drop=0.1):
        super().__init__()
        self.proj_e = Linear(edge_dim, hid)
        self.layers = torch.nn.ModuleList()
        last_dim = in_dim
        for _ in range(layers):
            self.layers.append(GATv2Conv(last_dim, hid//heads, heads=heads, edge_dim=hid, dropout=drop))
            last_dim = hid
        self.bn = BatchNorm1d(last_dim)
        self.act = ReLU()
        self.drop = Dropout(drop)

    def forward(self, x, edge_index, edge_attr, batch):
        e = self.proj_e(edge_attr) if edge_attr.numel() > 0 else torch.zeros((edge_index.size(1), self.layers[0].out_channels), device=x.device)
        h = x
        for gat in self.layers:
            h = gat(h, edge_index, e)
            h = F.elu(h)
        h = self.bn(h)
        h = self.act(h)
        h = self.drop(h)
        g = global_mean_pool(h, batch)
        return g

class GNNRegressor(Module):
    def __init__(self, in_dim, edge_dim, hid=128, layers=3, heads=2, drop=0.1, out_tasks=5):
        super().__init__()
        self.backbone = GATBackbone(in_dim, edge_dim, hid, layers, heads, drop)
        emb = hid
        self.trunk = Sequential(
            Linear(emb, emb), ReLU(), Dropout(drop),
            Linear(emb, emb//2), ReLU(), Dropout(drop),
        )
        self.heads = torch.nn.ModuleList([Linear(emb//2, 1) for _ in range(out_tasks)])

    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
        g = self.backbone(x, edge_index, edge_attr, batch)
        z = self.trunk(g)
        outs = [head(z) for head in self.heads]
        return torch.cat(outs, dim=1)

# Training utilities

Implements epoch loop with MSE loss per target ignoring NaNs via masks; AdamW optimizer; cosine schedule optional; early stopping on validation loss.

In [ ]:
# --- Cell 6: Training Utilities ---

def loss_with_mask(pred, target):
    # pred/target: [B, 5]
    mask = ~torch.isnan(target)
    diff = (pred[mask] - target[mask])**2
    return diff.mean() if diff.numel() > 0 else torch.tensor(0.0, device=pred.device)

def run_epoch(model, loader, optimizer=None):
    is_train = optimizer is not None
    total = 0.0
    for data in loader:
        data = data.to(device)
        pred = model(data)
        tgt = data.y if hasattr(data, "y") else None
        if tgt is None:
            continue
        loss = loss_with_mask(pred, tgt)
        if is_train:
            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            optimizer.step()
        total += loss.item() * data.num_graphs
    return total / max(1, len(loader.dataset))

def predict_model(model, loader):
    outs = []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            p = model(data)
            outs.append(p.detach().cpu().numpy())
    return np.vstack(outs)

# Cross‑validation and OOF 

Uses KFold with consistent SEED; for each fold, train on train_idx and validate on val_idx; record OOF predictions aligned to original indices; predict on test and average across folds.

In [ ]:
# --- Cell 7: KFold Training, OOF, and Test Predictions ---

FOLDS = 5
EPOCHS = 40
LR = 2e-3
WD = 1e-4

def build_loader(ds, idxs, shuffle, batch_size=BATCH_SIZE):
    subset = Subset(ds, idxs)
    return DataLoader(subset, batch_size=batch_size, shuffle=shuffle, num_workers=2)

kf = KFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
oof = np.full((len(full_train_ds), 5), np.nan, dtype=np.float32)
test_preds = np.zeros((len(test_ds), 5), dtype=np.float32)

for fold, (tr_idx, va_idx) in enumerate(kf.split(range(len(full_train_ds)))):
    print(f"\nFold {fold+1}/{FOLDS} | train={len(tr_idx)} val={len(va_idx)}")
    tr_loader = build_loader(full_train_ds, tr_idx, shuffle=True)
    va_loader = build_loader(full_train_ds, va_idx, shuffle=False)
    te_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

    sample_data = smiles_to_data("CC")  # to infer dims; safe placeholder
    in_dim = sample_data.x.size(1)
    edge_dim = sample_data.edge_attr.size(1) if sample_data.edge_attr.numel() > 0 else 6

    model = GNNRegressor(in_dim, edge_dim, hid=160, layers=3, heads=2, drop=0.15).to(device)
    opt = optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)

    best = 1e9
    patience, wait = 8, 0
    for epoch in range(1, EPOCHS+1):
        tr_loss = run_epoch(model, tr_loader, opt)
        va_loss = run_epoch(model, va_loader, None)
        if va_loss < best - 1e-5:
            best = va_loss
            wait = 0
            best_state = copy.deepcopy(model.state_dict())
        else:
            wait += 1
        if epoch % 5 == 0:
            print(f"Epoch {epoch:03d} | train {tr_loss:.5f} | val {va_loss:.5f}")
        if wait >= patience:
            break

    model.load_state_dict(best_state)
    # OOF
    va_loader = build_loader(full_train_ds, va_idx, shuffle=False)
    oof_preds = predict_model(model, va_loader)
    oof[va_idx] = oof_preds

    # Test preds
    fold_test = predict_model(model, te_loader)
    test_preds += fold_test / FOLDS

    del model, opt, tr_loader, va_loader, te_loader
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

# Export predictions

Saves OOF and test predictions as CSVs per target for downstream blending with the non‑GNN stack.

In [ ]:
# --- Cell 8: Export OOF and Test Predictions ---

oof_df = pd.DataFrame(oof, columns=targets)
oof_df.to_csv("gnn_oof.csv", index=False)

test_pred_df = pd.DataFrame(test_preds, columns=targets)
test_pred_df.insert(0, "id", test["id"])
test_pred_df.to_csv("gnn_test_preds.csv", index=False)

print("Saved: gnn_oof.csv and gnn_test_preds.csv")

# Optional: direct blend here

If desired, blend with the non‑GNN predictions and write a submission; otherwise, keep this notebook focused on producing GNN predictions.

In [ ]:
# --- Cell 9 (Optional): Blend with Non-GNN Stack ---

# If non-GNN predictions are available in the working directory:
if os.path.exists("stack_test_preds.csv"):
    stack = pd.read_csv("stack_test_preds.csv")  # columns: id + targets
    gnn  = pd.read_csv("gnn_test_preds.csv")
    blend = stack.copy()
    for t in targets:
        # Example 0.6 non-GNN / 0.4 GNN; adjust per CV
        blend[t] = 0.6 * stack[t].values + 0.4 * gnn[t].values
    blend.to_csv("submission.csv", index=False)
    print("submission.csv written via 0.6/0.4 blend.")

# Methodology

This GNN notebook complements the non‑GNN stacks by learning graph‑level representations from SMILES:

* Atom/bond featurization encodes chemistry structure; GATv2 captures local neighborhoods with edge‑aware message passing.

* Multi‑task heads predict Tg, FFV, Rg, Density, Tc jointly, with NaN‑masking in loss.

* 5‑fold CV provides OOF validation and robust test averaging.

* Exported predictions are designed for blending with the non‑GNN stack, typically at 30–50% weight depending on target CV.